### Import and install required packages and libraries

In [ ]:
# uncomment the applicable line to install (needed only once)

# For MacOs and Linux
# %pip install tk
# %pip install openpyxl
# %pip install pandas
# %pip install requests

# For Windows

In [ ]:
# import requiered libraries
import warnings
warnings.filterwarnings("ignore")
import tkinter
import csv
from csv import reader
import os
import json
from tkinter import filedialog
from tkinter import *
import pandas as pd
import requests
from zipfile import ZipFile
from pathlib import Path

### Select working directory

In [ ]:
def browse_button():
    # select a directory and store it in global var "folder_path"
    global folder_path
    filename = filedialog.askdirectory(initialdir='/Users/angel/Dropbox (Last Mile Team)/Mac/Desktop')
    folder_path.set(filename)
    print(filename)
    root.destroy()

root = Tk()
folder_path = StringVar()
lbl1 = Label(master=root,textvariable=folder_path)
lbl1.grid(row=0, column=1)
button2 = Button(text="Browse", command=browse_button)
button2.grid(row=0, column=3)

mainloop()
directory = os.path.join(folder_path.get())
os.chdir(directory)

### Read from lmt_LEAD_input_to_COPERT.xlsx

In [ ]:
df = pd.read_excel("lmt_LEAD_input_to_COPERT.xlsx", "URBAN_OFF_PEAK_SPEED")
urban_off_peak_speed = df.iloc[:,4]
df2 = pd.read_excel("lmt_LEAD_input_to_COPERT.xlsx", "URBAN_PEAK_SPEED")
urban_peak_speed = df2.iloc[:,4]
df3 = pd.read_excel("lmt_LEAD_input_to_COPERT.xlsx", "URBAN_PEAK_SHARE")
urban_peak_share = df3.iloc[:,4]
df4 = pd.read_excel("lmt_LEAD_input_to_COPERT.xlsx", "URBAN_OFF_PEAK_SHARE")
urban_off_peak_share = df4.iloc[:,4]

### Delete rows 2 to 12 of selected lmt_LEAD_input_to_COPERT.xlsx worksheets

In [ ]:
from openpyxl import load_workbook
 
# load excel file
workbook = load_workbook(filename="lmt_LEAD_input_to_COPERT.xlsx")
 
# open workbook
sheet = workbook.active

sheet_names = ["URBAN_OFF_PEAK_SPEED", "URBAN_PEAK_SPEED", "URBAN_OFF_PEAK_SHARE", "URBAN_PEAK_SHARE", "STOCK", "MEAN_ACTIVITY"]
for sheet in sheet_names:
    workbook[sheet].delete_rows(2, 12)
workbook.save(filename = "lmt_LEAD_input_to_COPERT.xlsx")

### Write to lmt_LEAD_input_to_EVCO2.json

In [ ]:
import re
import math
import json
l = directory.split("_")
if 'diesel' in l:
    category = 'Light Commercial Vehicles'
    fuel = 'Diesel'
    segment = 'N1-III'
    num = 161
    eurostandard = 'Euro 6 a/b/c'
elif 'hybrid' in l:
    category = 'Passenger Cars'
    fuel = 'Petrol Hybrid'
    segment = 'Large-SUV-Executive'
    eurostandard = 'Euro 6 a/b/c'
    num = 161
elif 'nv200' in l:
    category = 'HeavyCommercial Vehicles'
    fuel = 'Diesel'
    segment = 'N1-III'
    eurostandard = 'Euro 6 a/b/c'
    num = 100
for path,dirs,files in os.walk(directory):
    for file in files:
        if "lmt_LEAD_input_to_EVCO2_2.json" in file or "lmt_LEAD_input_to_EVCO_2.json" in file:
            dd = pd.read_json("lmt_LEAD_input_to_COPERT.json")
            dd2 = pd.read_json("lmt_LEAD_input_to_EVCO2_2.json")
            dd['ResponsePlanId'] = dd2['ResponsePlanId']
            dd['Category'] = category
            dd['Fuel'] = fuel
            dd['Segment'] = segment
            dd['EuroStandard'] = eurostandard
            for path,dirs,files in os.walk(directory):
                for file in files:
                    if file.endswith(".xml") and 'Response' in path:
                        with open(os.path.join(path, file), 'r') as f:
                            a = f.read()
                            f.close()
            xml_pattern = "(?:<NumberOfServices.*?>)(.*?)(?:<\\/NumberOfServices>)"
            val = re.findall(xml_pattern, a)[0]
            dd['Stock'] = math.ceil(int(val) / num)
            dd['MeanActivity'] = 60
            with open("lmt_LEAD_input_to_COPERT.json", "w") as ff:
                data = json.loads(dd.to_json(orient = "records"))
                ff.truncate()
                ff.write(json.dumps(data))
                ff.close()
        else:
            pass

### Write to lmt_LEAD_input_to_COPERT.json

In [ ]:
d = pd.read_json("lmt_LEAD_input_to_COPERT.json")
d = d.rename(columns = {"Stock":"2021", "MeanActivity":"2021_"})
d2 = d[["Category", "Fuel", "Segment", "EuroStandard"]]
d3 =  d[["Category", "Fuel", "Segment", "EuroStandard","2021"]]
d4 = d[["Category", "Fuel", "Segment", "EuroStandard","2021_"]]
d4 = d4.rename(columns = {"2021_":"2021"})
if len(urban_peak_speed) != d2.shape[0]:
    if len(urban_peak_speed) > d2.shape[0]:
        urban_peak_speed = urban_peak_speed[:d2.shape[0]]
        urban_off_peak_speed = urban_off_peak_speed[:d2.shape[0]]
        urban_peak_share = urban_peak_share[:d2.shape[0]]
        urban_off_peak_share = urban_off_peak_share[:d2.shape[0]]
    else:
        while len(urban_peak_speed) != d2.shape[0]:
            urban_peak_speed =   urban_peak_speed.append(pd.Series(urban_peak_speed[0]), ignore_index = True)
            urban_off_peak_speed = urban_off_peak_speed.append(pd.Series(urban_off_peak_speed[0]), ignore_index = True)
            urban_peak_share = urban_peak_share.append(pd.Series(urban_peak_share[0]), ignore_index = True)
            urban_off_peak_share = urban_off_peak_share.append(pd.Series(urban_off_peak_share[0]), ignore_index = True)

### Read from lmt_LEAD_input_to_COPERT.json

In [ ]:
sheets = ["URBAN_OFF_PEAK_SPEED", "URBAN_PEAK_SPEED", "URBAN_OFF_PEAK_SHARE", "URBAN_PEAK_SHARE"]
book = load_workbook('lmt_LEAD_input_to_COPERT.xlsx')
writer = pd.ExcelWriter('lmt_LEAD_input_to_COPERT.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}
for sheetname in writer.sheets:
    if sheetname == "URBAN_OFF_PEAK_SPEED":
        d2[df.columns[4]] = urban_off_peak_speed.values
        d2.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)
    elif sheetname == "URBAN_PEAK_SPEED":
        d2[df2.columns[4]] = urban_peak_speed.values
        d2.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)
    elif sheetname == "URBAN_OFF_PEAK_SHARE":
        d2[df3.columns[4]] = urban_off_peak_share.values
        d2.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)
    elif sheetname == "URBAN_PEAK_SHARE":
        d2[df4.columns[4]] = urban_peak_share.values
        d2.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)
    elif sheetname == "STOCK":
        d3.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)
    elif sheetname == "MEAN_ACTIVITY":
        d4.to_excel(writer,sheet_name=sheetname, startrow=0, index = False)

    else:
        pass

writer.save()

### Send lmt_LEAD_input_to_COPERT.xlsx to LEAD COPERT VM

In [ ]:
files = {
    'input_data': open(directory + '/lmt_LEAD_input_to_COPERT.xlsx', 'rb'),
    'keep ': (None, 'true'),
    'noevap': (None, 'false'),
    'alt_country ': (None, 'ES'),
    'EF': (None, 'true'),
    'EB': (None, 'true'),
    'outputFile': (None, 'xlsx'),
}

response = requests.post('http://3.120.144.230:8000', files=files)

with open(directory + '/Response_from_COPERT.zip', 'wb') as f:
    f.write(response.content)

### Download, store and clean LEAD COPERT VM response

In [ ]:
Path(directory + "/Response_from_COPERT").mkdir(parents=True, exist_ok=True)

In [ ]:
with ZipFile(directory + '/Response_from_COPERT.zip', 'r') as zip:
    zip.extractall(directory + "/Response_from_COPERT/")

In [ ]:
if os.path.exists(directory + '/Response_from_COPERT.zip'):
    os.remove(directory + '/Response_from_COPERT.zip')